<a href="https://colab.research.google.com/github/ShuqairABD/Dicom-2-png/blob/main/%D0%9C%D0%9E%D0%99%D0%95_DICOM_2_png.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Start**

In [ ]:
!pip install pydicom Pillow matplotlib ipywidgets

In [ ]:
!pip install -q jupyter_bbox_widget roboflow dataclasses-json supervision

In [ ]:
!pip install pydicom pillow matplotlib ipywidgets jupyter_bbox_widget

In [4]:
import pydicom
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from jupyter_bbox_widget import BBoxWidget
import base64
import io
from matplotlib.widgets import RectangleSelector
import zipfile
import os
from datetime import datetime



In [5]:
from google.colab import output
output.enable_custom_widget_manager()


### **folder**

In [6]:
IS_COLAB = True

if IS_COLAB:
    from google.colab import output
    output.enable_custom_widget_manager()


In [ ]:

# Create directory
output_dir = '/content/cropped_images'
os.makedirs(output_dir, exist_ok=True)

#  encode image as base64 (строка)
def encode_image(image_array):
    pil_image = Image.fromarray(image_array)

    buffered = io.BytesIO()
    pil_image.save(buffered, format="PNG")
    encoded = base64.b64encode(buffered.getvalue()).decode()
    return "data:image/png;base64," + encoded

# crop and save with new filenames
def crop_and_save_images(dicom_path):

    print(f"Loading DICOM file from {dicom_path}...")
    ds = pydicom.dcmread(dicom_path)
    print("DICOM file loaded.")

    # Extract pixel data
    pixel_array = ds.pixel_array
    print("Pixel data extracted.")

    # BBox
    print("Initializing BBoxWidget...")
    widget = BBoxWidget()
    widget.image = encode_image(pixel_array)
    print("BBoxWidget initialized.")

    # cropping and saving images
    def crop_image(widget):
        bboxes = widget.bboxes
        print(f"Widget bboxes: {bboxes}")
        if bboxes:
            for i, bbox in enumerate(bboxes):
                x_start, y_start, width, height = bbox['x'], bbox['y'], bbox['width'], bbox['height']
                x_end, y_end = x_start + width, y_start + height

                # Убедиться, что bbox допустимо
                if width > 0 and height > 0:
                    cropped_array = pixel_array[y_start:y_end, x_start:x_end]

                    # Save cropped image with a new filename
                    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                    filename = f'image_bbox_{timestamp}_{i}.png'
                    filepath = os.path.join(output_dir, filename)
                    Image.fromarray(cropped_array).save(filepath)
                    print(f"Cropped image saved as '{filepath}'")
                else:
                    print("Invalid bounding box dimensions.")

    #  bbox changes
    def on_bbox_change(change):
        print("Bounding box updated:", widget.bboxes)

    widget.observe(on_bbox_change, names='bboxes')

    # Кнопка
    crop_button = widgets.Button(description="Crop and Save Image")
    crop_button.on_click(lambda b: crop_image(widget))

    # Display widgets
    print("Displaying widgets...")
    display(widget, crop_button)
    print("Widgets displayed.")

# Обработать файлы DICOM в папке
dicom_folder = '/content/test'
dicom_files = [f for f in os.listdir(dicom_folder) if f.endswith('.dcm')]

for dicom_file in dicom_files:
    dicom_path = os.path.join(dicom_folder, dicom_file)
    print(f"Processing DICOM file: {dicom_path}...")
    crop_and_save_images(dicom_path)




### **zip**

In [8]:

folder_to_zip = '/content/cropped_images'
zip_file_path = '/content/cropped_images.zip'


with zipfile.ZipFile(zip_file_path, 'w') as zipf:

    for root, dirs, files in os.walk(folder_to_zip):
        for file in files:

            relative_path = os.path.relpath(os.path.join(root, file), folder_to_zip)
            zipf.write(os.path.join(root, file), relative_path)

print(f"ZIP file created and ready to download: {zip_file_path}")

from google.colab import files
files.download(zip_file_path)


ZIP file created and ready to download: /content/cropped_images.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>